In [1]:
# Import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
import numpy as np
import sqlite3


In [2]:
# create engine to historical prices sqlite db
engine = create_engine("sqlite:///test.sqlite")

In [3]:
eur_usd_df = pd.read_sql('select * from eur_usd',engine)
cad_usd_df = pd.read_sql('select * from cad_usd', engine)
jpy_usd_df = pd.read_sql('select * from jpy_usd', engine)

In [4]:
eur_usd_df

,Date,Open,High,Low,Close
0,01/01/25,1.0356,1.0364,1.0352,1.0359
1,12/31/24,1.0409,1.0426,1.0345,1.0355
2,12/30/24,1.0427,1.0459,1.0373,1.0408
3,12/27/24,1.0419,1.0446,1.0406,1.0428
4,12/26/24,1.0407,1.0432,1.0381,1.0425
...,...,...,...,...,...
258,01/05/24,1.0945,1.0998,1.0877,1.0939
259,01/04/24,1.0920,1.0975,1.0915,1.0946
260,01/03/24,1.0937,1.0967,1.0895,1.0922
261,01/02/24,1.1046,1.1056,1.0938,1.0940


In [5]:
# eur_usd_df[' Close']
def MA5(table):
    counter = 0
    avg5_list = []
    for x in table[' Close']:
        if(counter>4):
            avg5 = np.average([table[' Close'][counter - 5],
                               table[' Close'][counter - 4],
                               table[' Close'][counter - 3],
                               table[' Close'][counter - 2],
                               table[' Close'][counter - 1]])
            avg5_list.append(avg5)
        else:
            avg5_list.append(None)
        counter += 1

    return avg5_list


In [6]:
eur_usd_df[' MA5'] = MA5(eur_usd_df)
cad_usd_df[' MA5'] = MA5(cad_usd_df)
jpy_usd_df[' MA5'] = MA5(jpy_usd_df)
# print(jpy_usd_df[0:10])

In [7]:
def MA8(table):
    counter = 0
    avg8_list = []
    for x in table[' Close']:
        if(counter>7):
            avg8 = np.average([table[' Close'][counter - 8],
                               table[' Close'][counter - 7],
                               table[' Close'][counter - 6],
                               table[' Close'][counter - 5],
                               table[' Close'][counter - 4],
                               table[' Close'][counter - 3],
                               table[' Close'][counter - 2],
                               table[' Close'][counter - 1]])
            avg8_list.append(avg8)
        else:
            avg8_list.append(None)
        counter += 1

    return avg8_list

In [8]:
eur_usd_df[' MA8'] = MA8(eur_usd_df)
cad_usd_df[' MA8'] = MA8(cad_usd_df)
jpy_usd_df[' MA8'] = MA8(jpy_usd_df)
# jpy_usd_df[0:12]

In [9]:
def MA13(table):
    counter = 0
    avg13_list = []
    for x in table[' Close']:
        if(counter>12):
            avg13 = np.average([table[' Close'][counter - 13],
                                table[' Close'][counter - 12],
                                table[' Close'][counter - 11],
                                table[' Close'][counter - 10],
                                table[' Close'][counter - 9],
                                table[' Close'][counter - 8],
                                table[' Close'][counter - 7],
                                table[' Close'][counter - 6],
                                table[' Close'][counter - 5],
                                table[' Close'][counter - 4],
                                table[' Close'][counter - 3],
                                table[' Close'][counter - 2],
                                table[' Close'][counter - 1]])
            avg13_list.append(avg13)
        else:
            avg13_list.append(None)
        counter += 1

    return avg13_list

In [10]:
eur_usd_df[' MA13'] = MA13(eur_usd_df)
cad_usd_df[' MA13'] = MA13(cad_usd_df)
jpy_usd_df[' MA13'] = MA13(jpy_usd_df)
# jpy_usd_df[0:15]

In [11]:
def setup_buy_col(table):
        buy = []
        for row in table.iterrows():
                buy.append('Closed')
        return buy

def setup_sell_col(table):
        sell = []
        for row in table.iterrows():
                sell.append('Closed')
        return sell

def setup_prof_col(table):
        profit = []
        for row in table.iterrows():
                profit.append(0)
        return profit


In [12]:
eur_usd_df['Buy'] = setup_buy_col(eur_usd_df)
eur_usd_df['Sell'] = setup_sell_col(eur_usd_df)
eur_usd_df['Profit'] = setup_prof_col(eur_usd_df)

cad_usd_df['Buy'] = setup_buy_col(cad_usd_df)
cad_usd_df['Sell'] = setup_sell_col(cad_usd_df)
cad_usd_df['Profit'] = setup_prof_col(cad_usd_df)

jpy_usd_df['Buy'] = setup_buy_col(jpy_usd_df)
jpy_usd_df['Sell'] = setup_sell_col(jpy_usd_df)
jpy_usd_df['Profit'] = setup_prof_col(jpy_usd_df)

In [13]:
def algo_buy(table):
    for index, row in table.iterrows():
        if (index>12):
            if (table['Buy'][index-1] == 'Closed'): # in closed position, want to check if should open position
                if ((table[' MA5'][index] > table[' MA8'][index]) & (table[' MA8'][index] > table[' MA13'][index])):
                    table['Buy'][index] = 'Open'

            if (table['Buy'][index-1] == 'Open'): # in open position, want to check if should close position
                if (table[' Close'][index] < table[' MA13'][index]):
                    table['Buy'][index] = 'Closed'
                else:
                    table['Buy'][index] = 'Open'
    return table['Buy']

In [14]:
eur_usd_df['Buy'] = algo_buy(eur_usd_df)
cad_usd_df['Buy'] = algo_buy(cad_usd_df)
jpy_usd_df['Buy'] = algo_buy(jpy_usd_df)

/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/77332307.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Buy'][index] = 'Open'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/77332307.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Buy'][index] = 'Open'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/77332307.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t

In [15]:
def algo_sell(table):
    for index, row in table.iterrows():
        if (index>12):
            if (table['Sell'][index-1] == 'Closed'): # in closed position, want to check if should open position
                if ((table[' MA5'][index] < table[' MA8'][index]) & (table[' MA8'][index] < table[' MA13'][index])):
                    table['Sell'][index] = 'Open'

            if (table['Sell'][index-1] == 'Open'): # in open position, want to check if should close position
                if (table[' Close'][index] > table[' MA5'][index]):
                    table['Sell'][index] = 'Closed'
                else:
                    table['Sell'][index] = 'Open'
    return table['Sell']

In [16]:
eur_usd_df['Sell'] = algo_sell(eur_usd_df)
cad_usd_df['Sell'] = algo_sell(cad_usd_df)
jpy_usd_df['Sell'] = algo_sell(jpy_usd_df)

/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/2913671093.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Sell'][index] = 'Open'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/2913671093.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Sell'][index] = 'Closed'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/2913671093.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [17]:
eur_usd_df[58:70]

,Date,Open,High,Low,Close,MA5,MA8,MA13,Buy,Sell,Profit
58,10/11/24,1.0934,1.0956,1.0923,1.0938,1.08730,1.084550,1.083677,Open,Closed,0
59,10/10/24,1.0939,1.0957,1.0900,1.0938,1.08868,1.086500,1.084300,Open,Closed,0
60,10/09/24,1.0976,1.0984,1.0936,1.0940,1.09080,1.088225,1.085215,Open,Closed,0
61,10/08/24,1.0977,1.1000,1.0950,1.0981,1.09238,1.089762,1.086185,Open,Closed,0
62,10/07/24,1.0982,1.0990,1.0955,1.0976,1.09416,1.091162,1.087608,Open,Closed,0
63,10/04/24,1.1034,1.1041,1.0952,1.0977,1.09546,1.092963,1.088746,Open,Closed,0
64,10/03/24,1.1046,1.1079,1.1009,1.1033,1.09624,1.094413,1.090246,Open,Closed,0
65,10/02/24,1.1071,1.1084,1.1033,1.1044,1.09814,1.096175,1.092038,Open,Closed,0
66,10/01/24,1.1127,1.1146,1.1046,1.1067,1.10022,1.097837,1.093785,Open,Closed,0
67,09/30/24,1.1173,1.1211,1.1116,1.1137,1.10194,1.099450,1.095308,Open,Closed,0


In [18]:
for index, row in eur_usd_df.iterrows():
    print(row[' Close'])

1.0359
1.0355
1.0408
1.0428
1.0425
1.0411
1.04
1.0407
1.043
1.0362
1.0355
1.0493
1.0514
1.0502
1.0467
1.0497
1.053
1.0555
1.0569
1.059
1.0512
1.051
1.0498
1.0579
1.0554
1.0569
1.0491
1.0497
1.0427
1.0474
1.0546
1.0596
1.0598
1.054
1.0531
1.0564
1.0623
1.0655
1.0719
1.0803
1.0731
1.0932
1.0878
1.0835
1.0884
1.0857
1.0819
1.0814
1.0796
1.0828
1.0782
1.08
1.0817
1.0869
1.0832
1.0861
1.0892
1.0911
1.0938
1.0938
1.094
1.0981
1.0976
1.0977
1.1033
1.1044
1.1067
1.1137
1.1165
1.1178
1.1134
1.118
1.1113
1.1164
1.116
1.1119
1.1115
1.1135
1.1076
1.1077
1.1014
1.1019
1.1037
1.1085
1.1113
1.1083
1.1043
1.1073
1.105
1.1079
1.1119
1.1185
1.1162
1.1194
1.1113
1.1151
1.1131
1.1086
1.1029
1.0971
1.1014
1.0995
1.0933
1.0917
1.0918
1.0922
1.0932
1.0953
1.0913
1.0793
1.0825
1.0817
1.082
1.0857
1.0845
1.0841
1.0855
1.0892
1.0884
1.0897
1.0939
1.0899
1.0896
1.0908
1.0869
1.083
1.0813
1.0825
1.0839
1.0813
1.079
1.0747
1.074
1.0716
1.0704
1.068
1.0716
1.0736
1.0695
1.0701
1.0745
1.0741
1.0735
1.0704
1.0736
1.0

In [19]:
def profit(table):
    open_price = 0
    closed_price = 0
    # buy profit
    for index, row in table.iterrows():
        if (index>12):
            if ((table['Buy'][index] == 'Closed') & (table['Buy'][index - 1] == 'Closed')): # we're in a closed postion
                table['Profit'][index] = 0
            elif ((table['Buy'][index] == 'Closed') & (table['Buy'][index - 1] == 'Open')): # we just closed a postition
                closed_price = table[' Close'][index]
                table['Profit'][index] = closed_price - open_price
            elif ((table['Buy'][index] == 'Open') & (table['Buy'][index - 1] == 'Open')): # we're in an open postion
                table['Profit'][index] = 0
            elif ((table['Buy'][index] == 'Open') & (table['Buy'][index - 1] == 'Closed')): # we just opend a postion
                open_price = table[' Close'][index]

    open_price = 0
    closed_price = 0
    # sell profit
    for index, row in table.iterrows():
        if (index>12):
            if ((table['Sell'][index] == 'Closed') & (table['Sell'][index - 1] == 'Closed')): # we're in a closed postion
                table['Profit'][index] = 0 + table['Profit'][index]     # add to profit just incase it's not zero
            elif ((table['Sell'][index] == 'Closed') & (table['Sell'][index - 1] == 'Open')): # we just closed a postition
                closed_price = table[' Close'][index]
                table['Profit'][index] = (open_price - closed_price) + table['Profit'][index]
            elif ((table['Sell'][index] == 'Open') & (table['Sell'][index - 1] == 'Open')): # we're in an open postion
                table['Profit'][index] = 0 + table['Profit'][index]
            elif ((table['Sell'][index] == 'Open') & (table['Sell'][index - 1] == 'Closed')): # we just opend a postion
                open_price = table[' Close'][index]

    return table['Profit']

In [20]:
eur_usd_df['Profit'] = profit(eur_usd_df)
cad_usd_df['Profit'] = profit(cad_usd_df)
jpy_usd_df['Profit'] = profit(jpy_usd_df)

/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/1398871199.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Profit'][index] = 0
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/1398871199.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Profit'][index] = 0
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_60870/1398871199.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
conn = sqlite3.connect('test.sqlite')

In [22]:
eur_usd_df.to_sql('eur_usd', conn, if_exists='replace', index=False)
cad_usd_df.to_sql('cad_usd', conn, if_exists='replace', index=False)
jpy_usd_df.to_sql('jpy_usd', conn, if_exists='replace', index=False)

263

In [23]:
conn.close()